<img style="float: right;" src="http://www2.le.ac.uk/liscb1.jpg">  
# Leicester Institute of Structural and Chemical Biology: Python for Biochemists
# Reading and Writing CSV files

One of the most common ways that data are stored is as Comma Separated Values (csv) files.  CSV files have many advanteages over other types - the two most important are:  
1. They are human readable, so if you need to get the data back in 10 years any software can do it (unlike formats such as .xls)
2. They can be as large as you like (unlike formats such as .xls)

... and they also have one big disadvantage: for the amount of data they store, they take up a lot of space.  By the time the files get so big that you may have trouble storing them, you'll probably have problems with computer RAM as well.  

While there are many ways to read and write these, the three most common are presented below.  You should choose which to use based on what you're going to do with the data once you have it.

## Python Standard Library  
This method works for all data, and doesn't use any external libraries.

### Writing data  
Although we're probabaly going to read data more often than write, it's easier to write the data as we're in total control.  We'll start with some imports and creating some test data:

In [1]:
import csv
test_column_names = ['Column 1', 'Column B', 3]
test_data = [1, 2, 7]

Because there are so many different ways to read and write CSV files, python uses `readers` and `writers`.  The simple example should be sufficient for most cases.

In [2]:
with open('../tmp/psl_csv_file_default.csv', mode='w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(test_column_names)
    csv_writer.writerow(test_data)

Now we can 'cat' the file using a *notebook magic* to see what actually got written:

In [3]:
%cat ../tmp/psl_csv_file_default.csv

If we want to have more quotes or different delimeters, these are all possible:

In [4]:
with open('../tmp/psl_csv_file_fancy.csv', mode='w') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=';', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
    csv_writer.writerow(test_column_names)
    csv_writer.writerow(test_data)

%cat ../tmp/psl_csv_file_fancy.csv

We can write tab-separated values (tsv) if we want:

In [5]:
with open('../tmp/psl_tsv_file.tsv', mode='w') as tsv_file:
    tsv_writer = csv.writer(tsv_file, delimiter='\t')  # \t is the escape character for tab
    tsv_writer.writerow(test_column_names)
    tsv_writer.writerow(test_data)

%cat ../tmp/psl_tsv_file.tsv

We can even write white-space-separated if we want (note how the writer automatically handles qoutes for us):

In [6]:
with open('../tmp/psl_wssv_file.csv', mode='w') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=' ')
    csv_writer.writerow(test_column_names)
    csv_writer.writerow(test_data)

%cat ../tmp/psl_wssv_file.csv

Because the writer just needs something to be 'file like', we can write directly to compressed files if we wish (note we have to be careful about how we open the file):

In [7]:
import gzip
with gzip.open('../tmp/psl_csv_file.zip', mode='wt') as zip_file:  # gzip defaults to binary data, so we use 'wt' for text
    csv_writer = csv.writer(zip_file)
    csv_writer.writerow(test_column_names)
    csv_writer.writerow(test_data)

%cat ../tmp/psl_csv_file.zip

�3�|_�psl_csv_file.zip r��)��S0�q�0�t�y�u�t�y�    �� A�   

Finally, we can use a mapping (like a dict) to create our csv files:

In [8]:
data_dict = {'Column 1': 1,
             'Column B': 2,
             3: 7
             }
data = [data_dict,]

with open('../tmp/psl_csv_file_from_dict.csv', mode='w') as csv_file:
    csv_writer = csv.DictWriter(csv_file, fieldnames = data[0].keys())
    
    csv_writer.writeheader()
    for item in data:
        csv_writer.writerow(item)

%cat ../tmp/psl_csv_file_from_dict.csv

This can be a bit dangerous, though - you have to make sure that all the dictionaries have the same keys.

In [9]:
from collections import namedtuple
data_dict_1 = {'Column 1': 1,
               'Column B': 2,
               3: 7
               }
data_dict_2 = {'Column 1': 1,
               'Column C': 42
               }
data = [data_dict_1, data_dict_2]

with open('../tmp/psl_csv_file_from_dict.csv', mode='w') as csv_file:
    csv_writer = csv.DictWriter(csv_file, fieldnames = data[0].keys())
    
    csv_writer.writeheader()
    for item in data:
        csv_writer.writerow(item)

%cat ../tmp/psl_csv_file_from_dict.csv

ValueError: dict contains fields not in fieldnames: 'Column C'

### Reading data
#### List output
As you'd expect, reading the data works similarly to how writing the data does - except in this case we need to know what the format is or we risk getting junk.  By Default, each row comes as a list.

In [10]:
with open('../tmp/psl_csv_file_default.csv', mode='r') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        print(row)

['Column 1', 'Column B', '3']
['1', '2', '7']


In [11]:
# semicolon-separated values
with open('../tmp/psl_csv_file_fancy.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=';', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
    for row in csv_reader:
        print(row)

['Column 1', 'Column B', 3.0]
[1.0, 2.0, 7.0]


Notice that if you get the format wrong, you still get *something* out!

In [12]:
# semicolon-separated values
with open('../tmp/psl_csv_file_fancy.csv') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        print(row)

['Column 1;"Column B";3']
['1;2;7']


In [13]:
# tab-separated values
with open('../tmp/psl_tsv_file.tsv') as tsv_file:
    tsv_reader = csv.reader(tsv_file, delimiter='\t')  # \t is the escape character for tab
    for row in tsv_reader:
        print(row)

['Column 1', 'Column B', '3']
['1', '2', '7']


In [14]:
# space-separated values
with open('../tmp/psl_wssv_file.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=' ')
    for row in csv_reader:
        print(row)

['Column 1', 'Column B', '3']
['1', '2', '7']


In [15]:
# gziped file
with gzip.open('../tmp/psl_csv_file.zip', mode='rt') as zip_file:  # use 'rt' to get text
    csv_reader = csv.reader(zip_file)
    for row in csv_reader:
        print(row)

['Column 1', 'Column B', '3']
['1', '2', '7']


#### dict output
In the examples above, we'd have to interpret the first row as column headers ourselves.  It's also possible to read the csv files as dictionaries (this is often very convenient.)

In [16]:
with open('../tmp/psl_csv_file_default.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        for k, v in row.items():
            print(f'{k}: {v}')

Column 1: 1
Column B: 2
3: 7


## Pandas

### Writing data
Although we're probabaly going to read data more often than write, it's easier to write the data as we're in total control.  We'll start with some imports and creating some test data:

In [17]:
import pandas as pd

df  = pd.DataFrame([[1, 2, 7]], columns=['Column 1', 'Column B', 3])
df

,Column 1,Column B,3
0,1,2,7


One important difference here is that pandas adds an index, which shows up in the csv file.

In [18]:
df.to_csv('../tmp/pd_csv_default.csv')

%cat ../tmp/pd_csv_default.csv

,Column 1,Column B,3
0,1,2,7


If we specify one of the columns as an index, it all works as we'd expect...

In [19]:
df2 = pd.DataFrame([[1, 2, 7]], columns=['Column 1', 'Column B', 3]).set_index('Column 1')
df2

,Column B,3
Column 1,,
1,2,7


In [20]:
df2.to_csv('../tmp/pd2_csv_default.csv')

%cat ../tmp/pd2_csv_default.csv

Column 1,Column B,3
1,2,7


`Pandas` lets you do all the fancy formatting the `csv` module does - check the documentation if you need that.  Because `pandas` opens the file for you, if you want to use gzip compression, you have to specify that in the call. 

In [21]:
df2.to_csv('../tmp/pd2_csv_default.csv.gz', compression='gzip')

%cat ../tmp/pd2_csv_default.csv.gz

�:�|_�pd2_csv_default.csv r��)��S0�q�0�t��u�t̹    �� ��h   

### Reading Data
`Pandas` reads csv files as well as you'd expect.  Because it's expecting a table-like format, it will try to use the first row as headers.  It even automatically detects compressed files:

In [22]:
df = pd.read_csv('../tmp/pd2_csv_default.csv.gz')
df

,Column 1,Column B,3
0,1,2,7


It doesn't know which column to use as the index, so you have to tell it (if you want an index column)

In [23]:
pd.read_csv('../tmp/pd2_csv_default.csv.gz', index_col='Column 1')

,Column B,3
Column 1,,
1,2,7


You can turn off the automatic header detection if you like, which will then automatically generate header names.

In [24]:
pd.read_csv('../tmp/pd2_csv_default.csv.gz', header=None)

,0,1,2
0,Column 1,Column B,3
1,1,2,7


## Numpy  
Like all numpy based work, this is only really appropriate if you're going to be doing array-based numerical computing.

### Writing data
Although we're probabaly going to read data more often than write, it's easier to write the data as we're in total control.  We'll start with some imports and creating some test data:

In [25]:
import numpy as np
a = np.array([(1, 2, 3),
              (4, 5, 6)])
print(f' array \n{a}\n has type {a.dtype}')

 array 
[[1 2 3]
 [4 5 6]]
 has type int64


In [26]:
np.savetxt('../tmp/np_csv_default.csv', a)

%cat ../tmp/np_csv_same-dtype.csv

1.000000000000000000e+00 2.000000000000000000e+00 3.000000000000000000e+00
4.000000000000000000e+00 5.000000000000000000e+00 6.000000000000000000e+00


### Reading data

Reading data, again, is just the opposite of writing.  `Numpy` has loads of options to read csv files with all sorts of different formats and data types - but if you're not reading numeric arrays, you should probably be using `pandas`.

In [27]:
b = np.loadtxt('../tmp/np_csv_default.csv')
print(f' array \n{b}\n has type {b.dtype}')

 array 
[[1. 2. 3.]
 [4. 5. 6.]]
 has type float64


### Compressed Data
Note that `numpy` can read and write compressed data, but only it's binary format.

In [28]:
np.savez_compressed('../tmp/np_csv_default.npz', a)

%cat ../tmp/np_csv_default.npz

PK      ! �c�W   �   	  arr_0.npy  �       W       �����P�P���Z�\�n��n�i������_TR���_��
wK�)N�g$��F:
ƚ:
�
 PK      ! �c�W   �   	           �    arr_0.npyPK      7   �     

In [29]:
arrays = np.load('../tmp/np_csv_default.npz')
for array_name in arrays.files:
    print(arrays[array_name])

[[1 2 3]
 [4 5 6]]
